In [0]:

autoloader_df = (spark.readStream
                             .format('cloudFiles')
                             .option("cloudFiles.format", "json")
			                .option("cloudFiles.schemaLocation","/Volumes/gizmobox/landing/operational_data/schema/_schema")
                             .option("cloudFiles.inferColumnTypes", "true")
                             .option("cloudFiles.schemaEvolutionMode", "addNewColumns")
                             
                             .load("/Volumes/gizmobox/landing/operational_data/customers_autoloader/")

)


In [0]:
from pyspark.sql.functions import col, current_timestamp, row_number, split
autoloader_transform = (
    autoloader_df
    .withColumn("fname", split(col("customer_name"), " ").getItem(0))
    .withColumn("lname", split(col("customer_name"), " ").getItem(1))
    .withColumn("created_time", current_timestamp())
    .filter(col("customer_id").isNotNull())
)



In [0]:
autoloader_query = (autoloader_transform.writeStream
                .format("delta")
                .option("checkpointLocation","/Volumes/gizmobox/landing/operational_data/schema/checkpoint_autoloader")
                .option("mergeSchema", "true")
                .toTable("gizmobox.bronze.customer_autoloader")
                              
                )

In [0]:
%sql
select * from gizmobox.bronze.customer_autoloader order by customer_id

In [0]:
%sql 
drop table gizmobox.bronze.customer_autoloader